In [2]:
# Use centrailzed training to compare with federated learning
epochs = 30
n_train_items = 12800

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np

# import torchcsprng as csprng


In [4]:
torch.cuda.is_available()

True

In [6]:
class Arguments():
    def __init__(self):
        self.batch_size = 250
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.25
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# random_number_generator = csprng.create_random_device_generator(
#                 "/dev/urandom")

In [7]:
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

# train_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(train_loader)
#         if i < n_train_items / args.batch_size
#     ]


# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

In [13]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.5)
        optimizer.step()
        
        with torch.no_grad():
            for param in model.parameters():
                param.add_(torch.normal(0,1.3*1.5,param.size(),device=device)*args.lr/args.batch_size)
#                 param.add_(torch.from_numpy(np.random.normal(0,1.3*1.5,param.size())*args.lr).to(device))
        
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [14]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():

            
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.test_batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.test_batch_size),
        100. * correct / (len(test_loader)*args.test_batch_size)))


In [16]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322716
Train Epoch: 1 [7500/60000 (12%)]	Loss: 0.886326
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.361024
Train Epoch: 1 [22500/60000 (38%)]	Loss: 0.320207
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.169778
Train Epoch: 1 [37500/60000 (62%)]	Loss: 0.196953
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.133199
Train Epoch: 1 [52500/60000 (88%)]	Loss: 0.139904

Test set: Average loss: 0.0943, Accuracy: 9695/10048 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.106349
Train Epoch: 2 [7500/60000 (12%)]	Loss: 0.133910
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.050741
Train Epoch: 2 [22500/60000 (38%)]	Loss: 0.128230
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.098846
Train Epoch: 2 [37500/60000 (62%)]	Loss: 0.084215
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.098715
Train Epoch: 2 [52500/60000 (88%)]	Loss: 0.059285

Test set: Average loss: 0.0602, Accuracy: 9812/10048 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.048269
Train Epoch: 3 [7500/60000 (12%)]	Loss: 0.045